# Власні Значення та Вектори у Фінансах: Аналіз Ризиків Портфеля

## Eigenvalues and Eigenvectors in Finance: Portfolio Risk Analysis

---

У цьому notebook ми розглянемо практичне застосування власних значень (eigenvalues) та власних векторів (eigenvectors) для аналізу ризиків інвестиційного портфеля акцій.

## Блок 1: Вступ та Постановка Задачі

### Що ми хочемо зробити?

Уявімо, що ви портфельний менеджер і маєте інвестиції в декілька акцій. Вам потрібно зрозуміти:

1. **Які основні джерела ризику** впливають на ваш портфель?
2. **Як акції рухаються разом** — чи є приховані фактори, що впливають на всі?
3. **Як оптимізувати диверсифікацію** — знайти справді незалежні напрямки інвестування?

### Чому eigenvalues/eigenvectors корисні для цього?

Коваріаційна матриця дохідностей акцій описує, як акції рухаються відносно одна одної. Але ця матриця може бути складною для інтерпретації.

**Власні вектори** розкладають цю складну структуру на **незалежні напрямки ризику**:
- Кожен власний вектор = один незалежний "фактор ризику"
- Власне значення = величина ризику (варіації) в цьому напрямку

Це основа методу **Principal Component Analysis (PCA)**, який широко використовується в:
- Risk Management
- Factor Investing
- Portfolio Optimization

### Наші дані

Ми працюватимемо з **5 умовними акціями** з різних секторів:
- **TECH** — технологічна компанія
- **BANK** — банківський сектор
- **RETAIL** — роздрібна торгівля
- **ENERGY** — енергетичний сектор
- **HEALTH** — фармацевтика/охорона здоров'я

## Блок 2: Підготовка Даних

In [ ]:
# Імпортуємо необхідні бібліотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Налаштування для кращої візуалізації
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Фіксуємо seed для відтворюваності результатів
np.random.seed(42)

### Створення синтетичних даних дохідності акцій

Ми створимо реалістичні дані, де акції мають:
- **Спільний ринковий фактор** — коли ринок росте/падає, більшість акцій рухаються разом
- **Секторні фактори** — акції одного сектору корелюють сильніше
- **Індивідуальний шум** — унікальні рухи кожної акції

In [ ]:
# Параметри симуляції
n_days = 252  # Один торговий рік
n_stocks = 5
stock_names = ['TECH', 'BANK', 'RETAIL', 'ENERGY', 'HEALTH']

# Створюємо приховані фактори
# Фактор 1: Загальний ринковий фактор (впливає на всі акції)
market_factor = np.random.normal(0, 0.01, n_days)

# Фактор 2: "Growth vs Value" (tech і health vs bank і energy)
growth_factor = np.random.normal(0, 0.008, n_days)

# Фактор 3: Секторний (retail окремо)
sector_factor = np.random.normal(0, 0.006, n_days)

# Матриця навантажень факторів на акції (factor loadings)
# Кожен рядок — акція, кожен стовпець — фактор
factor_loadings = np.array([
    # Market, Growth, Sector
    [1.2,   0.8,   0.1],   # TECH: високий market beta, growth
    [1.0,  -0.6,   0.2],   # BANK: market, anti-growth (value)
    [0.9,   0.1,   0.9],   # RETAIL: market, unique sector
    [0.8,  -0.5,  -0.3],   # ENERGY: lower market, value
    [0.7,   0.4,  -0.2],   # HEALTH: defensive, growth
])

# Збираємо фактори в матрицю
factors = np.column_stack([market_factor, growth_factor, sector_factor])

# Генеруємо дохідності: systematic + idiosyncratic
systematic_returns = factors @ factor_loadings.T
idiosyncratic_returns = np.random.normal(0, 0.005, (n_days, n_stocks))

# Загальні дохідності
returns = systematic_returns + idiosyncratic_returns

# Створюємо DataFrame
dates = pd.date_range(start='2024-01-01', periods=n_days, freq='B')
returns_df = pd.DataFrame(returns, index=dates, columns=stock_names)

print("Перші 10 рядків даних дохідностей (daily returns):")
print(returns_df.head(10))
print(f"\nРозмір даних: {returns_df.shape[0]} днів × {returns_df.shape[1]} акцій")

### Чому використовуємо дохідності (returns), а не ціни?

**Важливо:** У фінансовому аналізі ми працюємо з **дохідностями**, а не з цінами акцій:

1. **Стаціонарність** — ціни мають тренд, дохідності коливаються навколо нуля
2. **Порівнянність** — акція за \$10 і за \$1000 мають однакові відсоткові зміни
3. **Статистичні властивості** — коваріаційна матриця має сенс для стаціонарних рядів

$$r_t = \frac{P_t - P_{t-1}}{P_{t-1}} \approx \ln\left(\frac{P_t}{P_{t-1}}\right)$$

In [ ]:
# Візуалізація часових рядів дохідностей
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Графік 1: Дохідності кожної акції
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B1F2B']
for i, stock in enumerate(stock_names):
    axes[0].plot(returns_df.index, returns_df[stock], 
                 label=stock, alpha=0.7, linewidth=0.8, color=colors[i])

axes[0].set_title('Щоденні дохідності акцій (Daily Returns)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Дата')
axes[0].set_ylabel('Дохідність (Return)')
axes[0].legend(loc='upper right')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Графік 2: Кумулятивні дохідності (щоб побачити загальний рух)
cumulative_returns = (1 + returns_df).cumprod() - 1
for i, stock in enumerate(stock_names):
    axes[1].plot(cumulative_returns.index, cumulative_returns[stock] * 100,
                 label=stock, linewidth=1.5, color=colors[i])

axes[1].set_title('Кумулятивні дохідності (Cumulative Returns)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('Кумулятивна дохідність (%)')
axes[1].legend(loc='upper left')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\nСтатистика дохідностей:")
print(returns_df.describe().round(4))

## Блок 3: Матриця Перетворення — Коваріаційна Матриця

### Що таке коваріаційна матриця?

Коваріаційна матриця $\Sigma$ описує **спільну варіабельність** між активами:

$$\Sigma = \begin{pmatrix} 
\sigma_1^2 & \text{cov}(r_1, r_2) & \cdots & \text{cov}(r_1, r_n) \\
\text{cov}(r_2, r_1) & \sigma_2^2 & \cdots & \text{cov}(r_2, r_n) \\
\vdots & \vdots & \ddots & \vdots \\
\text{cov}(r_n, r_1) & \text{cov}(r_n, r_2) & \cdots & \sigma_n^2
\end{pmatrix}$$

### Ключові властивості:

1. **Квадратна матриця** (n × n) — кількість активів визначає розмір
2. **Симетрична** — $\text{cov}(X, Y) = \text{cov}(Y, X)$
3. **Діагональні елементи** = дисперсія (variance) = волатильність²
4. **Недіагональні елементи** = коваріація (спільний рух активів)
5. **Позитивно напіввизначена** — всі власні значення ≥ 0

In [ ]:
# Обчислюємо коваріаційну матрицю
cov_matrix = returns_df.cov()

print("Коваріаційна матриця (Covariance Matrix):")
print("="*60)
print(cov_matrix.round(6))
print("\n")

# Перевіряємо властивості
print("Перевірка властивостей:")
print(f"  • Розмір: {cov_matrix.shape} (квадратна)")
print(f"  • Симетрична: {np.allclose(cov_matrix, cov_matrix.T)}")
print(f"  • Діагональ (дисперсії): {np.diag(cov_matrix).round(6)}")

In [ ]:
# Візуалізація коваріаційної матриці як heatmap
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Коваріаційна матриця
sns.heatmap(cov_matrix, annot=True, fmt='.5f', cmap='RdYlBu_r', 
            center=0, ax=axes[0], square=True,
            xticklabels=stock_names, yticklabels=stock_names)
axes[0].set_title('Коваріаційна матриця\n(Covariance Matrix)', fontsize=12, fontweight='bold')

# Кореляційна матриця (для порівняння)
corr_matrix = returns_df.corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0, vmin=-1, vmax=1, ax=axes[1], square=True,
            xticklabels=stock_names, yticklabels=stock_names)
axes[1].set_title('Кореляційна матриця\n(Correlation Matrix)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

### Інтерпретація матриць

**Коваріаційна матриця:**
- Діагональ показує волатильність кожної акції (TECH має найвищу)
- Позитивні недіагональні значення = акції рухаються в одному напрямку
- Масштаб залежить від волатильності активів

**Кореляційна матриця:**
- Нормалізована версія (значення від -1 до +1)
- Легше порівнювати силу зв'язку між парами
- 1 = ідеальна позитивна кореляція, -1 = негативна, 0 = незалежність

### Коли яку матрицю використовувати?

| Матриця | Використання |
|---------|-------------|
| **Коваріаційна** | Розрахунок ризику портфеля, PCA на сирих даних |
| **Кореляційна** | PCA коли активи мають різні масштаби, порівняння зв'язків |

## Блок 4: Обчислення Eigenvalues та Eigenvectors

### Математичне визначення

Для матриці $A$, вектор $\mathbf{v}$ є **власним вектором** з **власним значенням** $\lambda$, якщо:

$$A \mathbf{v} = \lambda \mathbf{v}$$

Тобто множення матриці на вектор лише **масштабує** цей вектор, не змінюючи напрямок.

### Для коваріаційної матриці це означає:
- **Власний вектор** = напрямок у просторі активів з незалежною варіацією
- **Власне значення** = величина дисперсії в цьому напрямку

In [ ]:
# Обчислюємо власні значення та вектори
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix.values)

print("Крок 1: Обчислення за допомогою numpy.linalg.eig()")
print("="*60)
print(f"\nВласні значення (eigenvalues): {eigenvalues.round(6)}")
print(f"\nВласні вектори (eigenvectors) — кожен стовпець:")
print(eigenvectors.round(4))

In [ ]:
# Перевірка: A @ v = λ * v
print("Крок 2: Перевірка рівняння A·v = λ·v")
print("="*60)

for i in range(len(eigenvalues)):
    v = eigenvectors[:, i]
    lam = eigenvalues[i]
    
    # Ліва частина: A @ v
    left_side = cov_matrix.values @ v
    
    # Права частина: λ * v
    right_side = lam * v
    
    # Перевірка
    is_equal = np.allclose(left_side, right_side)
    
    print(f"\nВласний вектор {i+1} (λ = {lam:.6f}):")
    print(f"  A·v = {left_side.round(6)}")
    print(f"  λ·v = {right_side.round(6)}")
    print(f"  Рівні: {is_equal} ✓" if is_equal else f"  Рівні: {is_equal} ✗")

In [ ]:
# Сортування за величиною eigenvalues (від найбільшого до найменшого)
print("Крок 3: Сортування за величиною власних значень")
print("="*60)

# Індекси для сортування
sorted_indices = np.argsort(eigenvalues)[::-1]  # Від більшого до меншого

# Сортуємо
eigenvalues_sorted = eigenvalues[sorted_indices]
eigenvectors_sorted = eigenvectors[:, sorted_indices]

# Обчислюємо explained variance ratio
total_variance = np.sum(eigenvalues_sorted)
explained_variance_ratio = eigenvalues_sorted / total_variance
cumulative_variance = np.cumsum(explained_variance_ratio)

print("\nРезультати (відсортовані):")
print("-"*60)
for i in range(len(eigenvalues_sorted)):
    print(f"PC{i+1}: λ = {eigenvalues_sorted[i]:.6f} | "
          f"Explained: {explained_variance_ratio[i]*100:.1f}% | "
          f"Cumulative: {cumulative_variance[i]*100:.1f}%")

In [ ]:
# Візуалізація: Scree Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot (eigenvalues)
x = range(1, len(eigenvalues_sorted) + 1)
axes[0].bar(x, eigenvalues_sorted, color='steelblue', alpha=0.8, edgecolor='navy')
axes[0].plot(x, eigenvalues_sorted, 'o-', color='darkred', linewidth=2, markersize=8)
axes[0].set_xlabel('Головна компонента (Principal Component)', fontsize=11)
axes[0].set_ylabel('Власне значення (Eigenvalue)', fontsize=11)
axes[0].set_title('Scree Plot: Власні значення', fontsize=13, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels([f'PC{i}' for i in x])

# Explained variance ratio
axes[1].bar(x, explained_variance_ratio * 100, color='forestgreen', alpha=0.8, 
            edgecolor='darkgreen', label='Individual')
axes[1].plot(x, cumulative_variance * 100, 'o-', color='darkred', 
             linewidth=2, markersize=8, label='Cumulative')
axes[1].axhline(y=80, color='gray', linestyle='--', alpha=0.7, label='80% threshold')
axes[1].set_xlabel('Головна компонента (Principal Component)', fontsize=11)
axes[1].set_ylabel('Пояснена дисперсія (%)', fontsize=11)
axes[1].set_title('Explained Variance Ratio', fontsize=13, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'PC{i}' for i in x])
axes[1].legend(loc='center right')
axes[1].set_ylim(0, 105)

plt.tight_layout()
plt.show()

print(f"\n📊 Висновок: Перші 2 компоненти пояснюють {cumulative_variance[1]*100:.1f}% загальної варіації!")

## Блок 5: Інтерпретація Власних Векторів

### Фінансовий сенс власних векторів

Кожен власний вектор представляє **портфель** — комбінацію акцій з певними вагами.

**Інтерпретація головних компонент у фінансах:**

| Компонента | Типова інтерпретація | Що показує |
|------------|---------------------|------------|
| **PC1** | Ринковий фактор (Market) | Загальний рух ринку, beta-ризик |
| **PC2** | Стиль (Growth vs Value) | Різниця між типами акцій |
| **PC3+** | Секторні/специфічні | Унікальні фактори ризику |

In [ ]:
# Створюємо DataFrame з власними векторами
eigenvectors_df = pd.DataFrame(
    eigenvectors_sorted,
    index=stock_names,
    columns=[f'PC{i+1}' for i in range(len(eigenvalues_sorted))]
)

print("Власні вектори (ваги акцій у кожній компоненті):")
print("="*60)
print(eigenvectors_df.round(3))

In [ ]:
# Візуалізація ваг акцій у перших трьох головних компонентах
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors_bars = ['#2E86AB' if x >= 0 else '#C73E1D' for x in eigenvectors_df['PC1']]

for idx, pc in enumerate(['PC1', 'PC2', 'PC3']):
    values = eigenvectors_df[pc].values
    colors_bars = ['#2E86AB' if x >= 0 else '#C73E1D' for x in values]
    
    bars = axes[idx].barh(stock_names, values, color=colors_bars, edgecolor='black', alpha=0.8)
    axes[idx].axvline(x=0, color='black', linewidth=0.8)
    axes[idx].set_xlabel('Вага (Loading)', fontsize=11)
    axes[idx].set_title(f'{pc}\n(λ = {eigenvalues_sorted[idx]:.5f}, {explained_variance_ratio[idx]*100:.1f}%)', 
                        fontsize=12, fontweight='bold')
    axes[idx].set_xlim(-1, 1)
    
    # Додаємо значення на графіку
    for bar, val in zip(bars, values):
        x_pos = val + 0.05 if val >= 0 else val - 0.05
        ha = 'left' if val >= 0 else 'right'
        axes[idx].text(x_pos, bar.get_y() + bar.get_height()/2, 
                      f'{val:.2f}', va='center', ha=ha, fontsize=10)

plt.suptitle('Ваги акцій у головних компонентах (Principal Component Loadings)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Інтерпретація результатів

**PC1 — Ринковий фактор:**
- Всі акції мають схожі за знаком ваги
- Це "загальний ринок" — коли PC1 росте, всі акції ростуть
- TECH має найбільшу вагу (найчутливіший до ринку)

**PC2 — Стильовий фактор:**
- Деякі акції мають позитивні, інші — негативні ваги
- Це протиставлення: наприклад, Tech/Health vs Bank/Energy
- Типово відображає "Growth vs Value" або секторну ротацію

**PC3+ — Специфічні фактори:**
- Пояснюють менше варіації
- Можуть відображати унікальні секторні або компанійні ризики

In [ ]:
# Візуалізація головних компонент як heatmap
fig, ax = plt.subplots(figsize=(10, 6))

sns.heatmap(eigenvectors_df, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, vmin=-1, vmax=1, ax=ax,
            cbar_kws={'label': 'Loading'})

ax.set_title('Factor Loadings Matrix\n(Як кожна акція "завантажена" на кожну компоненту)', 
             fontsize=13, fontweight='bold')
ax.set_xlabel('Головна компонента', fontsize=11)
ax.set_ylabel('Акція', fontsize=11)

plt.tight_layout()
plt.show()

In [ ]:
# Проекція дохідностей на головні компоненти (Principal Component Scores)
pc_scores = returns_df.values @ eigenvectors_sorted
pc_scores_df = pd.DataFrame(pc_scores, index=dates, 
                            columns=[f'PC{i+1}' for i in range(len(eigenvalues_sorted))])

# Візуалізація PC1 vs PC2
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot PC1 vs PC2
axes[0].scatter(pc_scores_df['PC1'], pc_scores_df['PC2'], 
                alpha=0.5, s=20, c='steelblue')
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].axvline(x=0, color='black', linewidth=0.5)
axes[0].set_xlabel('PC1 (Ринковий фактор)', fontsize=11)
axes[0].set_ylabel('PC2 (Стильовий фактор)', fontsize=11)
axes[0].set_title('Проекція дохідностей на PC1 vs PC2', fontsize=12, fontweight='bold')

# Часовий ряд PC1
axes[1].plot(pc_scores_df.index, pc_scores_df['PC1'], 
             color='steelblue', linewidth=0.8, label='PC1 (Market)')
axes[1].plot(pc_scores_df.index, pc_scores_df['PC2'], 
             color='#C73E1D', linewidth=0.8, alpha=0.7, label='PC2 (Style)')
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].set_xlabel('Дата', fontsize=11)
axes[1].set_ylabel('PC Score', fontsize=11)
axes[1].set_title('Часові ряди головних компонент', fontsize=12, fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

## Блок 6: Практичне Застосування в Індустрії

### 1. Risk Management: Ідентифікація головних джерел ризику

Замість аналізувати кореляції між усіма парами акцій, ми бачимо **кілька незалежних факторів ризику**.

In [ ]:
# Розподіл ризику по компонентах
print("Risk Decomposition (Розклад ризику)")
print("="*60)

total_portfolio_variance = np.sum(eigenvalues_sorted)
annual_portfolio_vol = np.sqrt(total_portfolio_variance * 252) * 100

print(f"\nЗагальна дисперсія портфеля: {total_portfolio_variance:.6f}")
print(f"Річна волатильність (annualized): {annual_portfolio_vol:.1f}%")
print("\nВнесок кожного фактора ризику:")
print("-"*60)

for i in range(len(eigenvalues_sorted)):
    component_vol = np.sqrt(eigenvalues_sorted[i] * 252) * 100
    print(f"  PC{i+1}: {explained_variance_ratio[i]*100:5.1f}% ризику | "
          f"Vol: {component_vol:5.1f}% річних")

### 2. Portfolio Optimization: Диверсифікація по головним компонентам

Традиційна диверсифікація (купити багато акцій) може бути ілюзорною, якщо всі акції рухаються від одного фактора.

**Справжня диверсифікація** = мінімізувати експозицію до домінуючих факторів.

In [ ]:
# Приклад: Побудова market-neutral портфеля
# Ідея: вибрати ваги так, щоб мінімізувати вплив PC1 (ринкового фактора)

# PC1 loadings
pc1_loadings = eigenvectors_sorted[:, 0]

print("Market-Neutral Portfolio Construction")
print("="*60)
print(f"\nLoadings на PC1 (ринковий фактор):")
for stock, loading in zip(stock_names, pc1_loadings):
    print(f"  {stock}: {loading:+.3f}")

# Простий підхід: long/short на основі PC2
pc2_loadings = eigenvectors_sorted[:, 1]

print(f"\nLoadings на PC2 (style factor):")
for stock, loading in zip(stock_names, pc2_loadings):
    direction = "LONG" if loading > 0 else "SHORT"
    print(f"  {stock}: {loading:+.3f} → {direction}")

print("\n💡 Інсайт: Портфель, побудований на PC2, буде:\n"
      "   - Long акції з позитивними loadings\n"
      "   - Short акції з негативними loadings\n"
      "   - Нейтральний до загального ринку (PC1)")

### 3. Factor Investing: Побудова факторних портфелів

In [ ]:
# Симуляція дохідностей факторних портфелів
fig, ax = plt.subplots(figsize=(12, 6))

# Кумулятивні дохідності перших 3 PC
for i, pc in enumerate(['PC1', 'PC2', 'PC3']):
    cumulative_pc = (1 + pc_scores_df[pc]).cumprod() - 1
    ax.plot(cumulative_pc.index, cumulative_pc * 100, 
            linewidth=1.5, label=f'{pc} Factor Portfolio')

# Equal-weighted portfolio для порівняння
equal_weight_returns = returns_df.mean(axis=1)
cumulative_eq = (1 + equal_weight_returns).cumprod() - 1
ax.plot(cumulative_eq.index, cumulative_eq * 100, 
        linewidth=2, linestyle='--', color='black', label='Equal-Weight Portfolio')

ax.axhline(y=0, color='gray', linewidth=0.5)
ax.set_xlabel('Дата', fontsize=11)
ax.set_ylabel('Кумулятивна дохідність (%)', fontsize=11)
ax.set_title('Factor Portfolio Performance', fontsize=13, fontweight='bold')
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()

### 4. Stress Testing: Аналіз чутливості

In [ ]:
# Stress test: Що станеться при шоку в 3 стандартних відхилення?
print("Stress Test Analysis")
print("="*60)

shock_size = 3  # 3 sigma event

print(f"\nСценарій: Шок у {shock_size} стандартних відхилення для кожного фактора\n")

for i in range(3):  # Перші 3 компоненти
    pc_std = np.sqrt(eigenvalues_sorted[i])
    shock_magnitude = shock_size * pc_std
    
    # Вплив на кожну акцію
    impact = eigenvectors_sorted[:, i] * shock_magnitude * 100
    
    print(f"PC{i+1} Shock ({explained_variance_ratio[i]*100:.1f}% variance):")
    print(f"  Magnitude: {shock_magnitude*100:.2f}%")
    print("  Impact on stocks:")
    for stock, imp in zip(stock_names, impact):
        print(f"    {stock}: {imp:+.2f}%")
    print()

### 5. Dimensionality Reduction: Зменшення кількості факторів

Замість відслідковувати 5 окремих акцій, можемо працювати з 2-3 факторами, які пояснюють більшість варіації.

In [ ]:
# Reconstruction error при використанні різної кількості компонент
print("Dimensionality Reduction Analysis")
print("="*60)

# Оригінальні дані
original_data = returns_df.values

reconstruction_errors = []

for n_components in range(1, len(eigenvalues_sorted) + 1):
    # Використовуємо перші n компонент
    selected_eigenvectors = eigenvectors_sorted[:, :n_components]
    
    # Проекція та реконструкція
    projected = original_data @ selected_eigenvectors
    reconstructed = projected @ selected_eigenvectors.T
    
    # Помилка реконструкції
    error = np.mean((original_data - reconstructed) ** 2)
    reconstruction_errors.append(error)
    
    print(f"  {n_components} компонент(и): "
          f"Explained = {cumulative_variance[n_components-1]*100:.1f}%, "
          f"MSE = {error:.8f}")

# Візуалізація
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(range(1, 6), reconstruction_errors, color='steelblue', alpha=0.8, edgecolor='navy')
ax.set_xlabel('Кількість компонент', fontsize=11)
ax.set_ylabel('Mean Squared Error', fontsize=11)
ax.set_title('Reconstruction Error vs Number of Components', fontsize=13, fontweight='bold')
ax.set_xticks(range(1, 6))
plt.tight_layout()
plt.show()

## Блок 7: Підсумки

### Ключові концепції

| Концепція | Математика | Фінансова інтерпретація |
|-----------|------------|------------------------|
| **Коваріаційна матриця** | $\Sigma = E[(X-\mu)(X-\mu)^T]$ | Як активи рухаються разом |
| **Власний вектор** | $Av = \lambda v$ | Напрямок незалежного ризику (фактор) |
| **Власне значення** | $\lambda$ | Величина дисперсії в напрямку фактора |
| **PC1** | Найбільше λ | Ринковий фактор (систематичний ризик) |
| **PC2+** | Менші λ | Стильові, секторні, специфічні фактори |

### Зв'язок з PCA

**Principal Component Analysis (PCA)** — це саме:
1. Обчислення коваріаційної (або кореляційної) матриці
2. Знаходження власних векторів та значень
3. Сортування за величиною власних значень
4. Вибір потрібної кількості компонент

Ми щойно реалізували PCA "вручну"!

### Де це використовується в індустрії

- **Barra**, **MSCI** — комерційні факторні моделі для risk management
- **Quantitative Trading** — статистичний арбітраж, pairs trading
- **Portfolio Construction** — risk parity, factor tilts
- **Derivatives Pricing** — моделювання кореляцій

### Додаткові ресурси

1. **"Quantitative Risk Management"** — McNeil, Frey, Embrechts
2. **"Factor Investing and Asset Allocation"** — Ang
3. **scikit-learn PCA documentation** — для готової реалізації

---

**Дякую за увагу!** 🎓

In [ ]:
# Бонус: Порівняння з sklearn PCA
from sklearn.decomposition import PCA

print("Verification: Порівняння з sklearn PCA")
print("="*60)

# PCA з sklearn
pca = PCA()
pca.fit(returns_df)

print("\nExplained variance ratio:")
print(f"  Our implementation: {explained_variance_ratio.round(4)}")
print(f"  sklearn PCA:        {pca.explained_variance_ratio_.round(4)}")

print("\n✅ Результати збігаються!")